In [1]:
from dataclasses import dataclass
from fractions import Fraction
import numpy as np

In [2]:
# round to nearest n division of a whole tone. 2 is usual (1/2), 4 is quarter-tone (1/4), etc.
def round_midi_to_microtone(x, microtone=2):
    microtone //= 2

    # base midi number
    base = round(x)
    # actual deviation
    dev = x - base

    # quantized deviation
    q_dev = round((dev * microtone * 100)/100)/microtone

    # np.float everywhere, looks ugly
    return base, str(Fraction(q_dev)/2), str(f'{dev*100:.2f}')

In [3]:
def midi_to_note_name(midi_number):
    sharp_note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    flat_note_names = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
    octave = midi_number // 12 - 1
    sharp_note = sharp_note_names[midi_number % 12]
    flat_note = flat_note_names[midi_number % 12]
    
    if sharp_note == flat_note:
        return f"{sharp_note}{octave}"
    else:
        return f"{sharp_note}{octave} / {flat_note}{octave}"

def note_name_to_midi(note_name):
    note_name = note_name.upper()
    # Mapping of note names to their semitone positions relative to C
    note_to_semitone = {
        'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3, 
        'E': 4, 'F': 5, 'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8, 
        'Ab': 8, 'A': 9, 'A#': 10, 'Bb': 10, 'B': 11
    }
    
    # Extract the note and octave from the input string
    note = note_name[:-1]
    octave = int(note_name[-1])
    
    # Calculate the MIDI number
    midi_number = 12 * (octave + 1) + note_to_semitone[note]
    
    return midi_number

def midi_to_hz(midi):
    return 440 * 2**((midi - 69) / 12)

def hz_to_midi(f):
    return 69 + 12 * np.log2(f / 440)

In [4]:
# f0 is the MIDI number of the fundamental frequency
# precision = how many divisions of a whole tone. 2 is 12-edo, 4 is quarter tone, etc.
# first N overtones or just give a nth overtone?
def overtone(f0_midi, n):
    hz = midi_to_hz(f0_midi)
    result = hz_to_midi(hz * n)

    return result

def nearest_midi_note(midi):
    return round(midi)

def nearest_microtone_adjustment(midi, microtone):
    microtone //= 2
    # base midi number
    base = round(midi)
    # actual deviation
    dev = midi - base
    # quantized deviation
    q_dev = round((dev * microtone * 100)/100)/microtone

    return Fraction(q_dev)/2

def reduce_octaves(midi):
    return midi % 12

In [5]:
@dataclass
class MicrotonalPitch:
    name: str
    microtone_deviation: Fraction
    cents_deviation: int

    def __repr__(self):
        return f'{self.name:<11} {str(self.microtone_deviation):>4} {self.cents_deviation:>3}'
    
    def __str__(self):
        return self.__repr__()

def float_midi_to_microtonal_pitch(midi, microtones):
    ob_midi = nearest_midi_note(midi)
    oc_dev = round((ob_midi - midi)*100)
    om_dev = nearest_microtone_adjustment(midi, microtones)
    note_name = midi_to_note_name(ob_midi)

    return MicrotonalPitch(note_name, om_dev, oc_dev)

def overtone_series(note, microtones, n_overtones):
    midi = note_name_to_midi(note)
    microtonal_pitches = []
    for i in range(1, n_overtones+1):
        o_midi = overtone(midi, i)
        microtonal_pitches.append(float_midi_to_microtonal_pitch(o_midi, microtones))

    return microtonal_pitches

In [6]:
# cents to sibelius pitch bend value
def cents_to_pitch_bend(cents):
    if cents <= 0:
        value = (1 + cents / 200) * 64
    else:
        value = (cents / 200) * (127 - 64) + 64

    return round(value)

In [7]:
# `up` is unnecessary. they are the same as the actual sounding notes
def _harmonic_location(n, up=True):
    if up:
        n = 1 / (1 - 1/n)
    
    return 12 * np.log2(n)

def harmonic_location(n, note, microtones, up=True):
    distance = _harmonic_location(n, up)
    midi = note_name_to_midi(note)
    position = distance + midi

    return float_midi_to_microtonal_pitch(position, microtones)

In [8]:
# filter odd overtones
my_overtones = overtone_series('d4', 8, 35)
for idx, val in zip(range(1, 35), my_overtones):
    if idx % 2 == 0:
        continue
    print(f'{idx:<2}: {val} - microtonal pitch bend: {cents_to_pitch_bend(val.microtone_deviation * 200)} - actual pitch bend: {cents_to_pitch_bend(val.cents_deviation)}')

1 : D4             0   0 - microtonal pitch bend: 64 - actual pitch bend: 64
3 : A5             0  -2 - microtonal pitch bend: 64 - actual pitch bend: 63
5 : F#6 / Gb6   -1/8  14 - microtonal pitch bend: 56 - actual pitch bend: 68
7 : C7          -1/8  31 - microtonal pitch bend: 56 - actual pitch bend: 74
9 : E7             0  -4 - microtonal pitch bend: 64 - actual pitch bend: 63
11: G#7 / Ab7   -1/4  49 - microtonal pitch bend: 48 - actual pitch bend: 79
13: A#7 / Bb7    1/4 -41 - microtonal pitch bend: 80 - actual pitch bend: 51
15: C#8 / Db8      0  12 - microtonal pitch bend: 64 - actual pitch bend: 68
17: D#8 / Eb8      0  -5 - microtonal pitch bend: 64 - actual pitch bend: 62
19: F8             0   2 - microtonal pitch bend: 64 - actual pitch bend: 65
21: G8          -1/8  29 - microtonal pitch bend: 56 - actual pitch bend: 73
23: G#8 / Ab8    1/8 -28 - microtonal pitch bend: 72 - actual pitch bend: 55
25: A#8 / Bb8   -1/8  27 - microtonal pitch bend: 56 - actual pitch bend: 73

In [21]:
for i in range(2, 11):
    print(i, harmonic_location(i, 'd3', 8), "    ", float_midi_to_microtonal_pitch(overtone(note_name_to_midi('d3'), i), 8))

2 D4             0   0      D4             0   0
3 A3             0  -2      A4             0  -2
4 G3             0   2      D5             0   0
5 F#3 / Gb3   -1/8  14      F#5 / Gb5   -1/8  14
6 F3           1/8 -16      A5             0  -2
7 F3          -1/8  33      C6          -1/8  31
8 E3           1/8 -31      D6             0   0
9 E3             0  -4      E6             0  -4
10 E3          -1/8  18      F#6 / Gb6   -1/8  14


###### def arithmetic_mean(*xs):
    xs = [Fraction(x) for x in xs]
    return sum(xs) / len(xs)

In [11]:
def harmonic_mean(*xs):
    xs = [Fraction(x) for x in xs]
    return len(xs) / sum(1/x for x in xs)

In [12]:
def divide_into(a, b, divisor):
    if a > b:
        a, b = b, a

    diff = b - a
    step = Fraction(diff / divisor)

    return [a + step * i for i in range(divisor + 1)]

In [13]:
def get_all_related_overtones(*xs, divisor=None):
    am = arithmetic_mean(*xs)
    hm = harmonic_mean(*xs)

    ams = f'Arithmetic mean: {am}'
    hms = f'Harmonic mean: {hm}'

    if len(xs) == 2 and divisor is not None:
        division = divide_into(*xs, divisor)
        division_s = f'Division: {[str(d) for d in division]}'
        return ams, hms, division_s

    return ams, hms

In [14]:
get_all_related_overtones(1, 19, divisor=4), get_all_related_overtones(3, 13, divisor=4)

(('Arithmetic mean: 10',
  'Harmonic mean: 19/10',
  "Division: ['1', '11/2', '10', '29/2', '19']"),
 ('Arithmetic mean: 8',
  'Harmonic mean: 39/8',
  "Division: ['3', '11/2', '8', '21/2', '13']"))

In [15]:
get_all_related_overtones(5, 11, divisor=4), get_all_related_overtones(7, 19, divisor=4)

(('Arithmetic mean: 8',
  'Harmonic mean: 55/8',
  "Division: ['5', '13/2', '8', '19/2', '11']"),
 ('Arithmetic mean: 13',
  'Harmonic mean: 133/13',
  "Division: ['7', '10', '13', '16', '19']"))

In [16]:
get_all_related_overtones(5, 19, divisor=4), get_all_related_overtones(7, 11, divisor=4)

(('Arithmetic mean: 12',
  'Harmonic mean: 95/12',
  "Division: ['5', '17/2', '12', '31/2', '19']"),
 ('Arithmetic mean: 9',
  'Harmonic mean: 77/9',
  "Division: ['7', '8', '9', '10', '11']"))

In [17]:
get_all_related_overtones(7, 13, divisor=4), get_all_related_overtones(1, 5, divisor=4)

(('Arithmetic mean: 10',
  'Harmonic mean: 91/10',
  "Division: ['7', '17/2', '10', '23/2', '13']"),
 ('Arithmetic mean: 3',
  'Harmonic mean: 5/3',
  "Division: ['1', '2', '3', '4', '5']"))

In [18]:
get_all_related_overtones(3, 19, divisor=4), get_all_related_overtones(1, 13, divisor=4)

(('Arithmetic mean: 11',
  'Harmonic mean: 57/11',
  "Division: ['3', '7', '11', '15', '19']"),
 ('Arithmetic mean: 7',
  'Harmonic mean: 13/7',
  "Division: ['1', '4', '7', '10', '13']"))

In [19]:
get_all_related_overtones(7, 19, divisor=4), get_all_related_overtones(3, 17, divisor=4)

(('Arithmetic mean: 13',
  'Harmonic mean: 133/13',
  "Division: ['7', '10', '13', '16', '19']"),
 ('Arithmetic mean: 10',
  'Harmonic mean: 51/10',
  "Division: ['3', '13/2', '10', '27/2', '17']"))

In [20]:
get_all_related_overtones(11, 19, divisor=4), get_all_related_overtones(13, 17, divisor=4)

(('Arithmetic mean: 15',
  'Harmonic mean: 209/15',
  "Division: ['11', '13', '15', '17', '19']"),
 ('Arithmetic mean: 15',
  'Harmonic mean: 221/15',
  "Division: ['13', '14', '15', '16', '17']"))

In [ ]:
get_all_related_overtones(7, 5, divisor=4), get_all_related_overtones(1, 13, divisor=4)